In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import os
import copy
import time
from torchvision import datasets, models, transforms

In [47]:
mean = np.array([0.5,0.5,0.5])
std = np.array([0.25,0.25,0.25])

In [48]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [49]:
data_dir = 'data/hymenoptera_data'

In [50]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

In [7]:
dataset_sizes = {x : len(image_datasets[x]) for x in ['train','val']}

In [8]:
class_names = image_datasets['train'].classes

In [9]:
print(class_names)

['ants', 'bees']


In [10]:
print(dataset_sizes)

{'train': 244, 'val': 153}


In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [12]:
print(device)

cuda:0


In [13]:
model = torchvision.models.resnet18(pretrained=True)

In [14]:
num_features = model.fc.in_features

In [15]:
print(num_features)

512


In [16]:
model.fc = nn.Linear(num_features,2)

In [17]:
model = model.to(device)

In [22]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size = 7, gamma = 0.1)

## Train Model

In [22]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i,(inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [52]:
# model.fc = nn.Linear(num_ftrs, 2)

# model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Decay LR by a factor of 0.1 every 7 epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=2)


Epoch 0/1
----------
train Loss: 0.6057 Acc: 0.6475
val Loss: 0.4379 Acc: 0.8824

Epoch 1/1
----------
train Loss: 0.4858 Acc: 0.7869
val Loss: 0.3365 Acc: 0.9216

Training complete in 0m 28s
Best val Acc: 0.921569


## Freezing Layer

In [53]:
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [54]:
num_features = model.fc.in_features

In [55]:
print(num_features)

512


In [56]:
model.fc = nn.Linear(num_features,2)

In [57]:
model = model.to(device)

In [58]:
criterion = nn.CrossEntropyLoss()

In [60]:
optimizer = torch.optim.SGD(model.fc.parameters(),lr = 0.01)

In [61]:
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=7,gamma = 0.1)

In [62]:
model = train_model(model,criterion,optimizer,lr_scheduler,num_epochs = 2)

Epoch 0/1
----------
train Loss: 0.9657 Acc: 0.6189
val Loss: 0.2348 Acc: 0.9216

Epoch 1/1
----------
train Loss: 0.7404 Acc: 0.7008
val Loss: 0.2288 Acc: 0.9412

Training complete in 0m 13s
Best val Acc: 0.941176
